In [1]:
%config Completer.use_jedi = False

In [2]:
import os
import sys
import time
import pandas as pd
import numpy as np
import scipy
from matplotlib import pyplot as plt
import pickle
from tqdm import tqdm
import datetime

from scripts.multiple_logging import setup_logger
from scripts.utils import convert_ids_to_ordered, MovingAverage
from scripts.mnap import compute_mnap
tqdm.pandas()

In [3]:
aspects = pd.read_csv('data/aspects.csv').set_index("aspect_id")
features = pd.read_csv('data/features.csv').set_index('feature_id')
organizations = pd.read_csv('data/organisations.csv').set_index('org_id')
reviews = pd.read_csv('data/reviews.csv')
rubrics = pd.read_csv('data/rubrics.csv').set_index('rubric_id')
test_users = pd.read_csv('data/test_users.csv').set_index('user_id')
users = pd.read_csv('data/users.csv').set_index('user_id')

/miniconda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


#### Preprocessing the reviews

In [4]:
reviews = reviews[reviews.rating.notna()]
reviews['rating'] = reviews['rating'].apply(int)
reviews['org_id'] = reviews['org_id'].astype(str)
reviews['user_id'] = reviews['user_id'].astype(str)
organizations.index = organizations.index.astype(str)
users.index = users.index.astype(str)
test_users.index = test_users.index.astype(str)
rubrics.index = rubrics.index.astype(str)
aspects.index = aspects.index.astype(str)
### DONT FORGET TO CONVERT OTHER DATAFRAMES TO STR

In [5]:
rated = reviews.groupby('user_id').rating.count()
good_users = rated[rated > 10].index
rated_orgs = reviews.groupby('org_id').rating.count()
good_orgs = rated_orgs[rated_orgs > 10].index

In [6]:
cleaned_reviews = reviews.query('user_id in @good_users and org_id in @good_orgs')

In [7]:
users_ordered, orgs_ordered, reviews_ordered = convert_ids_to_ordered(users, organizations, reviews)
n_users = len(users_ordered)
n_orgs = len(orgs_ordered)

In [8]:
orgs_rubrics = orgs_ordered[['rubrics_id']]
for i in range(len(rubrics)):
    orgs_rubrics[rubrics.rubric_name.iloc[i]] = orgs_rubrics['rubrics_id'].apply(lambda x: str(rubrics.index[i]) in x.split())
orgs_rubrics = orgs_rubrics.drop('rubrics_id', axis=1)
orgs_rubrics = orgs_rubrics.join(orgs_ordered['ordered_id']).set_index('ordered_id')

/tmp/ipykernel_35836/540280539.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orgs_rubrics[rubrics.rubric_name.iloc[i]] = orgs_rubrics['rubrics_id'].apply(lambda x: str(rubrics.index[i]) in x.split())


In [9]:
orgs_features = orgs_ordered[['features_id']].astype(str)
for i in range(len(features)):
    orgs_features[features.feature_name.iloc[i]] = orgs_features['features_id'].apply(lambda x: str(features.index[i]) in x.split())
orgs_features = orgs_features.drop('features_id', axis=1)
orgs_features = orgs_features.join(orgs_ordered['ordered_id']).set_index('ordered_id')

In [10]:
validation_split_day = 1050
train_reviews = reviews_ordered[reviews_ordered.ts < validation_split_day]
test_reviews = reviews_ordered[reviews_ordered.ts >= validation_split_day]

#### Convert to sparce matrix

In [11]:
import scipy.sparse as sp
last_user = max(reviews_ordered.ordered_id_user)

def sp_indeces(df):
    df['user_index'] = df['ordered_id_user']
    df['org_index'] = df['ordered_id_org'] + last_user + 1
    

sp_indeces(train_reviews)
sp_indeces(test_reviews)

/tmp/ipykernel_35836/2938850681.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['user_index'] = df['ordered_id_user']
/tmp/ipykernel_35836/2938850681.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['org_index'] = df['ordered_id_org'] + last_user + 1


In [12]:
def make_sparse(df):
    length = len(df)
    coo = sp.coo_matrix(
        ([1] * 2 * length, 
        (
            list(range(length)) + list(range(length)),
            list(df['org_index']) + list(df['user_index'])
        )),
        shape=(length, n_users+n_orgs)
    )
    sparse_rubrics = sp.coo_matrix(df.join(orgs_rubrics, on='ordered_id_org').loc[:, 'Булочная, пекарня':].astype(float))
    sparse_features = sp.coo_matrix(df.join(orgs_features, on='ordered_id_org').loc[:, 'karaoke':].astype(float))
    return sp.hstack([coo, sparse_rubrics, sparse_features])
    
X_train = make_sparse(train_reviews)
X_test = make_sparse(test_reviews)
y_train = train_reviews['rating']
y_test = test_reviews['rating']

In [13]:
os.makedirs("sparse_data/", exist_ok=True)

In [14]:
import scipy.sparse as sp
sp.save_npz('sparse_data/X_train.npz', X_train)
sp.save_npz('sparse_data/X_test.npz', X_test)

In [15]:
y_train.to_csv('sparse_data/y_train.csv', index=False)
y_test.to_csv('sparse_data/y_test.csv', index=False)

## Generate data for submission run

In [16]:
test_users = test_users.join(users_ordered)
msk_mask = np.array(orgs_ordered.city == 'msk')

In [17]:
orgs_ordered['other_city'] = orgs_ordered['city'].apply(lambda x: 'spb' if x == 'msk' else 'msk')

In [ ]:
submission_reviews = []
for i in tqdm(range(len(test_users) // 100 + 1)):
    submission_review_batch = test_users.iloc[i*100:(i+1)*100].merge(
        orgs_ordered[['other_city', 'ordered_id']], how='inner',
        left_on='city', right_on='other_city', suffixes=('_user', '_org'))
    sp_indeces(submission_review_batch)
    submission_reviews.append(make_sparse(submission_review_batch))

  6%|▌         | 10/170 [01:01<16:07,  6.05s/it]

In [ ]:
os.makedirs("sparse_data/submission_batches/", exist_ok=True)

In [ ]:
for i in tqdm(range(len(submission_reviews))):
    sp.save_npz(f'sparse_data/submission_batches/batch{i}.npz', submission_reviews[i])